|![福建师范大学](E:\\Desktop\\学业\\第五学期课业\\数据分析\\Lab\\image.png)|
|:--:|
|课程: 数据分析|
|实验名称: |
|教师: 林劼|
|专业: 软件工程|
|班级: 21软工1班|
|姓名: --|
|学号: --|
|实验日期: |

# 实验目的

# 实验内容

# 实验环境

* 硬件环境
   * 设备名称 Shelter-R7000
   * CPU AMD Ryzen 5 5600H 3.30 GHz
   * GPU NVIDIA GeForce RTX 3050 Laptop
   * RAM 13.9 GB
* 系统环境
   * OS Windows 10 家庭中文版
   * 版本号 21H1
* 软件环境
   * Anaconda3-2023.07-2-Windows-x86_64
   * Jupyter Notebook
   * Visual Studio Code 1.82.2

# 实验步骤



（描述实验步骤及中间的结果或现象。在实验中做了什么事情，怎么做的，发生的现象和中间结果）



# 课程作业任务

## 任务一：下载相应数据的财务报表数据，计算相应的基本面指标（参见附录3.1）

### 下载数据网址https://data.eastmoney.com/

### 各组选择一个行业，每个行业对应多只证券

本组为1组, 选择钢铁行业.共36只证券

### 下载各只证券（一个行业30只证券）的财务报表：主要指标、资产负债表、利润表、现金流量表

In [ ]:
import baostock as bs
import pandas as pd


START_DATE = '2023-09-01'
END_DATE = '2023-09-28'
INDUSTRY = '钢铁'
PATH_DATA = "../Data/"
PATH_STOCK_INDUSTRY = PATH_DATA + "stock_industry.csv"
PATH_K_DATA = PATH_DATA + "k_data.csv"
PATH_PROFIT_DATA = PATH_DATA + "profit_data.csv"
PATH_OPERATION_DATA = PATH_DATA + "operation_data.csv"
PATH_GROWTH_DATA = PATH_DATA + "growth_data.csv"
PATH_BALANCE_DATA = PATH_DATA + "balance_data.csv"
PATH_CASH_FLOW_DATA = PATH_DATA + "cash_flow_data.csv"


def login():
    # 登陆系统
    lg = bs.login()

    # 显示登陆返回信息
    print('login respond error_code:' + lg.error_code)
    print('login respond  error_msg:' + lg.error_msg)
    

# 读取目标行业的股票代码列表
def search_code():
    # 读取股票行业数据，这里假设已有一个包含股票代码和行业信息的CSV文件
    code_df = pd.read_csv(PATH_STOCK_INDUSTRY, encoding='gbk')

    # 筛选出特定行业
    industry_df = code_df.loc[code_df['industry'] == INDUSTRY]

    # 获取行业内所有股票的代码列表
    industry_code_list = industry_df['code'].tolist()
    return industry_code_list


# 通过api查询
def query_api_data(codes, api_func, args, path):
    datas = []
    result = pd.DataFrame()
    
    # 下载5年内季度财报
    if not api_func == bs.query_history_k_data_plus:
        for code in codes:
            for year in range(2018, 2024):
                for quarter in range(1, 5):
                    result = api_func(code=code, year=year, quarter=quarter)
                    print(f'{api_func} respond error_code:'+result.error_code)
                    print(f'{api_func} respond  error_msg:'+result.error_msg)
                    while (result.error_code == '0') & result.next():
                        datas.append(result.get_row_data())
        result_df = pd.DataFrame(datas, columns=result.fields)
        result_df.to_csv(path, index=False)
        return
    
    # 下载日k线数据
    for code in codes:
        result = api_func(code=code, **args)
        print(f'{api_func} respond error_code:'+result.error_code)
        print(f'{api_func} respond  error_msg:'+result.error_msg)
        while (result.error_code == '0') & result.next():
            datas.append(result.get_row_data())

    result_df = pd.DataFrame(datas, columns=result.fields)
    result_df.to_csv(path, index=False)


if __name__ == '__main__':
    login()
    codes = search_code()

    # 查询历史A股K线数据
    query_api_data(codes, api_func=bs.query_history_k_data_plus, args={"fields": "date,code,open,high,low,close,preclose,volume,amount,adjustflag,turn,tradestatus,pctChg,isST", "start_date": START_DATE, "end_date": END_DATE, "frequency": "d", "adjustflag": "3"},  path=PATH_K_DATA)
    # 查询盈利能力数据
    query_api_data(codes, api_func=bs.query_profit_data, args={}, path=PATH_PROFIT_DATA)
    # 查询营运能力数据...
    query_api_data(codes, api_func=bs.query_operation_data, args={}, path=PATH_OPERATION_DATA)
    # 查询成长能力数据
    query_api_data(codes, api_func=bs.query_growth_data, args={}, path=PATH_GROWTH_DATA)
    # 查询偿债能力数据
    query_api_data(codes, api_func=bs.query_balance_data, args={}, path=PATH_BALANCE_DATA)
    # 查询现金流量数据
    query_api_data(codes, api_func=bs.query_cash_flow_data, args={}, path=PATH_CASH_FLOW_DATA)

    # 登出系统
    bs.logout()

### 按照3.1.2的公式计算基本面指标（以2023年9.30数据为基础计算，股价按照2023.9.28的收盘价计算）

计算行业内30只证券的基本面指标，如资产负债表、利润表、现金流量表中无法提供指标公式中数据，请在实验报告中说明缺什么数据。

In [11]:
import pandas as pd

# 文件路径
PATH_DATA = "../Data/"
PATH_K_DATA = PATH_DATA + "k_data.csv"
PATH_PROFIT_DATA = PATH_DATA + "profit_data.csv"
PATH_OPERATION_DATA = PATH_DATA + "operation_data.csv"
PATH_GROWTH_DATA = PATH_DATA + "growth_data.csv"
PATH_BALANCE_DATA = PATH_DATA + "balance_data.csv"
PATH_CASH_FLOW_DATA = PATH_DATA + "cash_flow_data.csv"

# 读取数据
k_data = pd.read_csv(PATH_K_DATA, encoding='gbk')
profit_data = pd.read_csv(PATH_PROFIT_DATA, encoding='gbk')
operation_data = pd.read_csv(PATH_OPERATION_DATA, encoding='gbk')
growth_data = pd.read_csv(PATH_GROWTH_DATA, encoding='gbk')
balance_data = pd.read_csv(PATH_BALANCE_DATA, encoding='gbk')
cash_flow_data = pd.read_csv(PATH_CASH_FLOW_DATA, encoding='gbk')


# 计算市盈率
k_data['peTTM'].info()

# 计算市净率
k_data['pbMRQ'].info()

# 计算资产收益率
profit_data['roeAvg'].info()

# 计算负债比率
balance_data['liabilityToAsset'].info()

# 计算现金流量比率
cash_flow_data['CFOToNP'].info()

# 计算每股盈利
profit_data['epsTTM'].info()

# 计算营业收入增长率
profit_data['MBRevenue'].info()
profit_data['netProfit'].info()
profit_data['gpMargin'].info()
profit_data['npMargin'].info()
balance_data['assetToEquity'].info()
balance_data['currentRatio'].info()
balance_data['quickRatio'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 720 entries, 0 to 719
Series name: peTTM
Non-Null Count  Dtype  
--------------  -----  
720 non-null    float64
dtypes: float64(1)
memory usage: 5.8 KB
<class 'pandas.core.series.Series'>
RangeIndex: 720 entries, 0 to 719
Series name: pbMRQ
Non-Null Count  Dtype  
--------------  -----  
720 non-null    float64
dtypes: float64(1)
memory usage: 5.8 KB
<class 'pandas.core.series.Series'>
RangeIndex: 805 entries, 0 to 804
Series name: roeAvg
Non-Null Count  Dtype  
--------------  -----  
794 non-null    float64
dtypes: float64(1)
memory usage: 6.4 KB
<class 'pandas.core.series.Series'>
RangeIndex: 808 entries, 0 to 807
Series name: liabilityToAsset
Non-Null Count  Dtype  
--------------  -----  
804 non-null    float64
dtypes: float64(1)
memory usage: 6.4 KB
<class 'pandas.core.series.Series'>
RangeIndex: 808 entries, 0 to 807
Series name: CFOToNP
Non-Null Count  Dtype  
--------------  -----  
726 non-null    float64
dtypes: float64(1)
me

# 实验结果与讨论

（描述最终得到的结果，并进行分析说明，可能的误差原因）

# 总结

（说明实验过程中遇到的问题及解决办法；新发现或个人的收获；未解决/需进一步研讨的问题或建议新实验方法等）